In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import griddata
import warnings
warnings.filterwarnings("ignore")
import os
import seaborn as sns

%matplotlib qt

In [3]:

#read data and rename cols
os.chdir('C:/Users/piago/Documents/Uni/5_Semester/Hiwi')
csv_file = "C:/Users/piago/Documents/Uni/5_Semester/Hiwi/Data/HE614_All_Grids_Single_Sheet.xlsx"

# Read the Excel file
df = pd.read_excel(csv_file, skiprows=10)
df = df.set_index('Date_Time')
df.index = pd.to_datetime(df.index)
df.rename(columns={'CTD_Temperature_SML' : 'CTD_Temperature_1',
                   'CTD_Temperature_30cm' : 'CTD_Temperature_30',
                   'CTD_Temperature_40cm' : 'CTD_Temperature_40',
                   'CTD_Temperature_50cm_[°C]' : 'CTD_Temperature_50',
                   'CTD_Temperature_60cm_[°C]': 'CTD_Temperature_60',
                   'CTD_Temperature_85cm' : 'CTD_Temperature_85',
                   'CTD_Temperature_100cm' : 'CTD_Temperature_100',
                   'RBR_Temperature_30cm' :  'RBR_Temperature_30',
                   'RBR_Temperature_40cm' :  'RBR_Temperature_40',
                   'RBR_Temperature_50cm' :  'RBR_Temperature_50',
                   'RBR_Temperature_60cm' :  'RBR_Temperature_60',
                   'RBR_Temperature_85cm' :  'RBR_Temperature_85',
                   'RBR_Temperature_100cm' :  'RBR_Temperature_100'
                   }, inplace= True)

df['Date'] = df.index.date
df_backup = df
# extract dfs
df_2 = df.loc['2023-03-02']
df_2 = df_2[(df_2.index < '2023-03-02 09:48') | (df_2.index >= '2023-03-02 09:49')]

df_3 = df.loc['2023-03-03']

df_8 = df.loc['2023-03-08']
df_8 = df_8[(df_8.index < '2023-03-08 09:19:10') | (df_8.index >= '2023-03-08 09:21:40')]


df_9 = df.loc['2023-03-09']
df_9.drop(df_9[df_9['CTD_Temperature_1']>6.2].index, inplace = True)


df_10 = df.loc['2023-03-10']

df_11 = df.loc['2023-03-11']

df_17 = df.loc['2023-03-17']
df_17 = df_17[df_17.index > '2023-03-17 08:03']

df_18 = df.loc['2023-03-18']
df_18 = df_18[df_18.index > '2023-03-18 07:50']

df_19 = df.loc['2023-03-19']
df_19 = df_19[(df_19.index < '2023-03-08 10:15:30') | (df_19.index >= '2023-03-19 10:16:20')]

df_20 = df.loc['2023-03-20']


#Detrend CTD, RBR and Campbell Air temp where reasonable


In [4]:
# correct temperatures
for df in [df_2, df_8, df_9, df_17]:
    # time in total seconds
    df['time_numeric'] = (df.index - df.index.min()).total_seconds()  

    # same for all depths
    if len(df) == len(df_8):
        depths = [1, 30, 40, 50, 60, 85]
    else:
        depths = [1, 30, 40, 50, 60, 85, 100]

    for d in depths:
        if df[f'CTD_Temperature_{d}'].dropna().empty:
            # If the column is empty (all NaNs), skip this depth
            print(f"{df['Date'][0]}: Column CTD_Temperature_{d} contains only NaN values. Creating nan-column for depth {d}.")
            df[f'detrend_2_Temp_{d}'] = np.nan

        else:
            coefficients = np.polyfit(df['time_numeric'], df[f'CTD_Temperature_{d}'], 2) # fit polynomial
            polynomial = np.poly1d(coefficients) #create function
            df[f'fitted_2_Temp_{d}'] = polynomial(df['time_numeric']) #values
            df[f'detrend_2_Temp_{d}'] = df[f'CTD_Temperature_{d}'] - df[f'fitted_2_Temp_{d}']
            df[f'detrend_2_Temp_{d}'] = df[f'detrend_2_Temp_{d}'] + df[f'fitted_2_Temp_{d}'].median()
            df.rename(columns={f'CTD_Temperature_{d}' : f'CTD_Temperature_{d}_old'}, inplace=True)
            df.rename(columns={f'detrend_2_Temp_{d}' : f'CTD_Temperature_{d}'}, inplace=True)


    # ... and same for airtemp
    coefficients = np.polyfit(df['time_numeric'], df['Campbell_Air_Temperature'], 2) # fit polynomial
    polynomial = np.poly1d(coefficients) #create function
    df[f'fitted_2_Temp_Air'] = polynomial(df['time_numeric']) #values
    df[f'Air_detrend_2_Temp'] = df['Campbell_Air_Temperature'] - df[f'fitted_2_Temp_Air']
    df[f'Air_detrend_2_Temp'] = df[f'Air_detrend_2_Temp'] + df[f'fitted_2_Temp_Air'].median()

    # ... and for RBRs
    depths = [30, 40, 50, 60, 85, 100]
    for d in depths:
        if df[f'RBR_Temperature_{d}'].dropna().empty:
            print(f"{df['Date'][0]}: Column RBR_Temperature_{d}cm is empty or contains only NaN values. creating nan-column for depth {d}.")
        else: 
            coefficients = np.polyfit(df['time_numeric'], df[f'RBR_Temperature_{d}'], 2) # fit polynomial
            polynomial = np.poly1d(coefficients) #create function
            df[f'rbr_fitted_2_Temp_{d}'] = polynomial(df['time_numeric']) #values
            df[f'rbr_detrend_2_Temp_{d}'] = df[f'RBR_Temperature_{d}'] - df[f'rbr_fitted_2_Temp_{d}']
            df[f'rbr_detrend_2_Temp_{d}'] = df[f'rbr_detrend_2_Temp_{d}'] + df[f'rbr_fitted_2_Temp_{d}'].median()

    print(f'Mission {str(df['Date'][0])} detrended.\n')
df_9_1 = df_9[df_9['time_numeric'] < (df_9['time_numeric'].max()/2)]
df_9_2 = df_9[df_9['time_numeric'] > (df_9['time_numeric'].max()/2)]

all_df = [df_2, df_3, df_8, df_9, df_10, df_11, df_17, df_18, df_19, df_20]


2023-03-02: Column CTD_Temperature_40 contains only NaN values. Creating nan-column for depth 40.
Mission 2023-03-02 detrended.

2023-03-08: Column CTD_Temperature_50 contains only NaN values. Creating nan-column for depth 50.
2023-03-08: Column RBR_Temperature_50cm is empty or contains only NaN values. creating nan-column for depth 50.
Mission 2023-03-08 detrended.

2023-03-09: Column CTD_Temperature_50 contains only NaN values. Creating nan-column for depth 50.
Mission 2023-03-09 detrended.

2023-03-17: Column CTD_Temperature_50 contains only NaN values. Creating nan-column for depth 50.
Mission 2023-03-17 detrended.



In [13]:
# look at anomaly only
for df in all_df:
    df['CTD_1_anomaly'] = df['CTD_Temperature_1'] - df['CTD_Temperature_1'].mean()
    df['CTD_30_anomaly'] = df['CTD_Temperature_30'] - df['CTD_Temperature_30'].mean()
    df['RBR_30_anomaly'] = df['RBR_Temperature_30'] - df['RBR_Temperature_30'].mean()
    df['Air_anomaly'] = df['Campbell_Air_Temperature'] - df['Campbell_Air_Temperature'].mean()
    # df['Air_smoothed_anomaly'] = df['Campbell_Airtemp_smoothed'] - df['Campbell_Airtemp_smoothed'].mean()


In [18]:
df = df_17

min_value = min(df['Air_anomaly'].min(), df['CTD_1_anomaly'].min())  # Get the minimum value for both axes
max_value = max(df['Air_anomaly'].max(), df['CTD_1_anomaly'].max())  # Get the maximum value for both axes

plt.scatter(df['Air_anomaly'], df['CTD_1_anomaly'])
plt.plot([min_value, max_value], [min_value, max_value], color='red', linestyle='--', label='x = y')
plt.title('Anomalies Air temperaute and SML temperature', fontsize = 18)
plt.xlabel('Air temperature anomaly', fontsize = 15)
plt.ylabel('SML temperature anomaly', fontsize = 15)
plt.xticks(size = 15)
plt.yticks(size = 15)
plt.show

<function matplotlib.pyplot.show(*, block=None)>